<a href="https://colab.research.google.com/github/vgrozov512-osp/PythonUrbanITMO2023_GrozovVA/blob/main/PythonUrban_final_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Итоговый проект. Улицы. Объекты культурного наследия.

In [1]:
# TODO собрать установку всех необходимых модулей в одном месте
!pip install osmnx
!pip install geopandas
!pip install mapclassify


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 kB 1.9 MB/s eta 0:00:00


In [2]:
# TODO собрать импорты всех модулей в одном месте
import osmnx as ox
import geopandas as gpd

In [3]:
TILES = "CartoDB positron"  # Название подложки для карт

# TODO указать любой район Санкт-Петербург из OSM https://wiki.openstreetmap.org/wiki/RU:%D0%A1%D0%B0%D0%BD%D0%BA%D1%82-%D0%9F%D0%B5%D1%82%D0%B5%D1%80%D0%B1%D1%83%D1%80%D0%B3/%D0%A0%D0%B0%D0%B9%D0%BE%D0%BD%D1%8B
TERRITORY_NAME = 'Калининский район, Санкт-Петербург'  # название территории для которой будут строиться слои

# TODO указать ссылку на файл из вашего github репозитория, которая начинается с https://raw.githubusercontent.com/
KGIOP_FILE_URL = "https://raw.githubusercontent.com/aeksei/PythonUrbanITMO2023/main/geojson_layers/kgiop_objects.geojson"  # ссылка на слой с объектами культурного наследия
STREETS_FILE_URL = "https://raw.githubusercontent.com/aeksei/PythonUrbanITMO2023/main/geojson_layers/streets.geojson"  # ссылка на слой с улицами

## Территория

### Загрузка территории из OSM (Extract)

In [9]:
# TODO загрузить geodataframe с геометрией для территории TERRITORY_NAME
district = ox.geocode_to_gdf('Калининский район, Санкт-Петербург')
district #вывод информации о районе (координаты, тип, наименование, геометрия и т.д.). Сработает, если закомментировать следующую команду.

district.explore(tiles="CartoDB positron") #отображение района на карте

## Улицы

### Загрузка файла с улицами (Extract)

In [10]:
# TODO отфильтровать улицы по маске геометрии территории полученной ранее
file_url = "https://raw.githubusercontent.com/aeksei/PythonUrbanITMO2023/main/geojson_layers/streets.geojson"

district = ox.geocode_to_gdf('Калининский район, Санкт-Петербург')
gdf = gpd.read_file(file_url, mask=district)
gdf

,name,geometry
0,проспект Металлистов,"LINESTRING (3382971.239 8393975.910, 3382946.0..."
1,улица Карпинского,"LINESTRING (3386426.663 8400785.882, 3386439.6..."
2,Брюсовская улица,"LINESTRING (3386897.867 8396690.882, 3386910.8..."
3,улица Карпинского,"LINESTRING (3386482.100 8400818.592, 3386603.4..."
4,Меншиковский проспект,"LINESTRING (3386719.834 8397463.093, 3386684.0..."
...,...,...
990,Светлановский проспект,"LINESTRING (3384444.330 8409666.286, 3384458.5..."
991,Боткинская улица,"LINESTRING (3379065.985 8390345.643, 3379084.3..."
992,Гражданский проспект,"LINESTRING (3383552.516 8402053.053, 3383508.7..."
993,аллея Академика Глушко,"LINESTRING (3380910.426 8405232.109, 3380854.5..."


### Обработка данных с улицами (Transform)

In [14]:
# TODO сгруппировать и объединить геометрии с одинаковыми названиями
routes_graph = ox.graph_from_place(TERRITORY_NAME, network_type='drive')
streets = ox.graph_to_gdfs(routes_graph, nodes=False)
streets["name"] = streets.loc[streets['name'].apply(lambda x: isinstance(x, list)), "name"].str.join(", ")
streets = streets.dissolve(by="name")
streets

,geometry,osmid,oneway,lanes,highway,maxspeed,reversed,length,junction,width,bridge
name,,,,,,,,,,,
"Бугровский путепровод, проспект Культуры","LINESTRING (30.37915 60.04974, 30.38016 60.050...","[129389224, 129389225, 737146607, 15579580, 39...",True,"[3, 2]",secondary,"[40, RU:urban]",False,772.713,None,None,yes
"Гражданский мост, Гражданский проспект","MULTILINESTRING ((30.41022 60.02568, 30.41009 ...","[183892195, 450106502, 130705737, 194972629, 3...",True,"[4, 2]",secondary,RU:urban,False,409.601,None,None,yes
"Кондратьевский проспект, площадь Калинина","LINESTRING (30.38643 59.97027, 30.38654 59.970...","[1217355659, 127434773]",True,2,secondary,RU:urban,False,25.282,None,None,None
"Кушелевская дорога, Кушелевский путепровод","MULTILINESTRING ((30.37336 59.98976, 30.37403 ...","[1185946234, 25453163, 154186379]",True,3,primary,RU:urban,False,577.321,None,None,yes
"Лабораторный проспект, проспект Мечникова","MULTILINESTRING ((30.40166 59.98698, 30.40138 ...","[1185928265, 1185928266, 31388770, 346882566]",False,"[4, 2]","[unclassified, residential]",RU:urban,True,658.113,None,None,None
"Литейный мост, улица Академика Лебедева","LINESTRING (30.34999 59.95345, 30.35001 59.953...","[653715820, 173298370, 653715819, 221309604]",True,3,primary,RU:urban,False,40.551,None,None,yes
"Меншиковский проспект, Нартовская улица","MULTILINESTRING ((30.42708 59.98854, 30.42618 ...","[346569138, 296082889, 4458858, 346569140]",False,"[4, 2]",residential,"[40, RU:urban]",True,407.352,None,None,None
"Пискарёвский проспект, Пискарёвский путепровод","LINESTRING (30.42990 59.98853, 30.42965 59.988...","[31402017, 348102882, 19841110, 374181631]",True,"[4, 3]",primary,RU:urban,False,343.050,None,None,yes
"Пискарёвский проспект, улица Руставели","LINESTRING (30.43720 59.99888, 30.43752 59.997...","[406220335, 707216086, 422006712, 1187612122, ...",True,3,primary,RU:urban,False,442.053,None,None,None


### Сохранение слоя с улицами (Load)

In [20]:
# TODO переименовать столбцы в русские названия, кроме столбца geometry
rename_columns = {"name": "Имя",
                  "name": "Имя",
                  "oneway": "Односторонняя",
                  "lanes": "Переулки",
                  "highway": "Статус дороги",
                  "maxspeed": "Максимальная допустимая скорость",
                  "reversed": "Обратная",
                  "length": "Протяжённость",
                  "junction": "Перекрёсток",
                  "width": "Ширина",
                  "bridge": "Мост"
                  }
streets.rename(columns=rename_columns, inplace=True)
streets
# TODO для того чтобы переименовать индекс, нужно обратиться и нему и вызвать от него метод rename (https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Index.rename.html)

,geometry,osmid,Односторонняя,Переулки,Статус дороги,Максимальная допустимая скорость,Обратная,Протяжённость,Перекрёсток,Ширина,Мост
name,,,,,,,,,,,
"Бугровский путепровод, проспект Культуры","LINESTRING (30.37915 60.04974, 30.38016 60.050...","[129389224, 129389225, 737146607, 15579580, 39...",True,"[3, 2]",secondary,"[40, RU:urban]",False,772.713,None,None,yes
"Гражданский мост, Гражданский проспект","MULTILINESTRING ((30.41022 60.02568, 30.41009 ...","[183892195, 450106502, 130705737, 194972629, 3...",True,"[4, 2]",secondary,RU:urban,False,409.601,None,None,yes
"Кондратьевский проспект, площадь Калинина","LINESTRING (30.38643 59.97027, 30.38654 59.970...","[1217355659, 127434773]",True,2,secondary,RU:urban,False,25.282,None,None,None
"Кушелевская дорога, Кушелевский путепровод","MULTILINESTRING ((30.37336 59.98976, 30.37403 ...","[1185946234, 25453163, 154186379]",True,3,primary,RU:urban,False,577.321,None,None,yes
"Лабораторный проспект, проспект Мечникова","MULTILINESTRING ((30.40166 59.98698, 30.40138 ...","[1185928265, 1185928266, 31388770, 346882566]",False,"[4, 2]","[unclassified, residential]",RU:urban,True,658.113,None,None,None
"Литейный мост, улица Академика Лебедева","LINESTRING (30.34999 59.95345, 30.35001 59.953...","[653715820, 173298370, 653715819, 221309604]",True,3,primary,RU:urban,False,40.551,None,None,yes
"Меншиковский проспект, Нартовская улица","MULTILINESTRING ((30.42708 59.98854, 30.42618 ...","[346569138, 296082889, 4458858, 346569140]",False,"[4, 2]",residential,"[40, RU:urban]",True,407.352,None,None,None
"Пискарёвский проспект, Пискарёвский путепровод","LINESTRING (30.42990 59.98853, 30.42965 59.988...","[31402017, 348102882, 19841110, 374181631]",True,"[4, 3]",primary,RU:urban,False,343.050,None,None,yes
"Пискарёвский проспект, улица Руставели","LINESTRING (30.43720 59.99888, 30.43752 59.997...","[406220335, 707216086, 422006712, 1187612122, ...",True,3,primary,RU:urban,False,442.053,None,None,None


In [21]:
# TODO сохранить слой в географической проекции в формате GeoJSON
gdf.to_crs(4326).to_file('streets.geojson', driver='GeoJSON')

## Объекты культурного наследия

### Загрузка объектов культурного наследия (Extract)

In [22]:
# TODO отфильтровать улицы по маске геометрии территории полученной ранее
file_url = "https://raw.githubusercontent.com/aeksei/PythonUrbanITMO2023/main/geojson_layers/kgiop_objects.geojson"

district = ox.geocode_to_gdf('Калининский район, Санкт-Петербург')
gdf = gpd.read_file(file_url, mask=district)
gdf

,id,ensemble_name,object_name,occurrence_time,object_location,historical_category,normative_act,object_type,geometry
0,2353,—,Пожарная часть,1930,"Лесной пр., 17",выявленный объект культурного наследия,Приказ председателя КГИОП № 15 от 20.02.2001,Памятник,POINT (3378699.369 8392081.702)
1,2354,—,Народный дом Нобеля,—,"Лесной пр., 19",объект культурного наследия регионального знач...,Закон Санкт-Петербурга № 141-47 от 02.07.1997,Памятник,POINT (3378678.458 8392221.495)
2,2230,Комплекс Санкт-Петербургской одиночной тюрьмы ...,Прачечная и ледник,1885-1890,"Арсенальная наб., 7; Комсомола ул., 8",выявленный объект культурного наследия,Приказ председателя КГИОП № 15 от 20.02.2001,Памятник,POINT (3379975.413 8389295.482)
3,2180,—,"Квартира, в которой жили в 1924-1934 гг. химик...",—,"Академика Лебедева ул., 10-г, кв. 4",объект культурного наследия регионального знач...,Закон Санкт-Петербурга № 174-27 от 05.07.1999,Памятник,POINT (3379068.019 8391163.126)
4,2181,—,Доходный дом Российского общества страхования ...,1912-1914,"Академика Лебедева ул., 12",выявленный объект культурного наследия,Приказ председателя КГИОП № 15 от 20.02.2001,Памятник,POINT (3379054.432 8391248.345)
...,...,...,...,...,...,...,...,...,...
264,9447,Комплекс зданий Агрофизического научно-исследо...,Главное здание,—,"Гражданский пр., 14, лит. А",выявленный объект культурного наследия,Распоряжение КГИОП от 31.05.2016 № 10-218,Памятник,POINT (3382321.835 8399879.935)
265,9448,Комплекс зданий Агрофизического научно-исследо...,Лаборатория свето-физиологии с оранжереей,—,"Гражданский пр., 14, лит. Д",выявленный объект культурного наследия,Распоряжение КГИОП от 31.05.2016 № 10-218,Памятник,POINT (3382290.777 8399731.656)
266,9534,Кондратьевский жилмассив,Жилые здания,1929-1931,"Санкт-Петербург , Кондратьевский проспект, 40",объект культурного наследия регионального знач...,Распоряжение КГИОП № 10-89 от 27.02.2014,Памятник,POINT (3382123.686 8392992.569)
267,9535,Кондратьевский жилмассив,Здание Калининского универмага,1929-1931,"Санкт-Петербург , Кондратьевский проспект, 40",объект культурного наследия регионального знач...,Распоряжение КГИОП № 10-89 от 27.02.2014,Памятник,POINT (3382476.012 8393048.401)


### Обработка объектов культурного наследия (Transform)

In [24]:
# TODO добавить два столбца lon и lat, в которых будут долгота и широта
osm_id = "R1114806"
territory = ox.geocode_to_gdf([osm_id], by_osmid=True)
gdf["lon"] = gdf.to_crs(3857).geometry.centroid.to_crs(4326).x
gdf["lat"] = gdf.to_crs(3857).geometry.centroid.to_crs(4326).y
gdf

,id,ensemble_name,object_name,occurrence_time,object_location,historical_category,normative_act,object_type,geometry,lon,lat
0,2353,—,Пожарная часть,1930,"Лесной пр., 17",выявленный объект культурного наследия,Приказ председателя КГИОП № 15 от 20.02.2001,Памятник,POINT (3378699.369 8392081.702),30.351373,59.965594
1,2354,—,Народный дом Нобеля,—,"Лесной пр., 19",объект культурного наследия регионального знач...,Закон Санкт-Петербурга № 141-47 от 02.07.1997,Памятник,POINT (3378678.458 8392221.495),30.351185,59.966222
2,2230,Комплекс Санкт-Петербургской одиночной тюрьмы ...,Прачечная и ледник,1885-1890,"Арсенальная наб., 7; Комсомола ул., 8",выявленный объект культурного наследия,Приказ председателя КГИОП № 15 от 20.02.2001,Памятник,POINT (3379975.413 8389295.482),30.362836,59.953064
3,2180,—,"Квартира, в которой жили в 1924-1934 гг. химик...",—,"Академика Лебедева ул., 10-г, кв. 4",объект культурного наследия регионального знач...,Закон Санкт-Петербурга № 174-27 от 05.07.1999,Памятник,POINT (3379068.019 8391163.126),30.354684,59.961463
4,2181,—,Доходный дом Российского общества страхования ...,1912-1914,"Академика Лебедева ул., 12",выявленный объект культурного наследия,Приказ председателя КГИОП № 15 от 20.02.2001,Памятник,POINT (3379054.432 8391248.345),30.354562,59.961847
...,...,...,...,...,...,...,...,...,...,...,...
264,9447,Комплекс зданий Агрофизического научно-исследо...,Главное здание,—,"Гражданский пр., 14, лит. А",выявленный объект культурного наследия,Распоряжение КГИОП от 31.05.2016 № 10-218,Памятник,POINT (3382321.835 8399879.935),30.383914,60.000638
265,9448,Комплекс зданий Агрофизического научно-исследо...,Лаборатория свето-физиологии с оранжереей,—,"Гражданский пр., 14, лит. Д",выявленный объект культурного наследия,Распоряжение КГИОП от 31.05.2016 № 10-218,Памятник,POINT (3382290.777 8399731.656),30.383635,59.999972
266,9534,Кондратьевский жилмассив,Жилые здания,1929-1931,"Санкт-Петербург , Кондратьевский проспект, 40",объект культурного наследия регионального знач...,Распоряжение КГИОП № 10-89 от 27.02.2014,Памятник,POINT (3382123.686 8392992.569),30.382134,59.969689
267,9535,Кондратьевский жилмассив,Здание Калининского универмага,1929-1931,"Санкт-Петербург , Кондратьевский проспект, 40",объект культурного наследия регионального знач...,Распоряжение КГИОП № 10-89 от 27.02.2014,Памятник,POINT (3382476.012 8393048.401),30.385299,59.969940


### Сохранение слоя с объектами культурного наследия (Load)

In [25]:
# TODO переименовать столбцы в русские названия, кроме столбца geometry
rename_columns = {"ensemble_name": "Название ансамбля",
                  "object_name": "Название объекта",
                  "occurrence_time": "Годы основания",
                  "object_location": "Расположение объекта",
                  "historical_category": "Историческая категория",
                  "normative_act": "Нормативный акт",
                  "object_type": "Тип объекта",
                  "lon": "Долгота",
                  "lat": "Широта",
                  }
gdf.rename(columns=rename_columns, inplace=True)
gdf

,id,Название ансамбля,Название объекта,Годы основания,Расположение объекта,Историческая категория,Нормативный акт,Тип объекта,geometry,Долгота,Широта
0,2353,—,Пожарная часть,1930,"Лесной пр., 17",выявленный объект культурного наследия,Приказ председателя КГИОП № 15 от 20.02.2001,Памятник,POINT (3378699.369 8392081.702),30.351373,59.965594
1,2354,—,Народный дом Нобеля,—,"Лесной пр., 19",объект культурного наследия регионального знач...,Закон Санкт-Петербурга № 141-47 от 02.07.1997,Памятник,POINT (3378678.458 8392221.495),30.351185,59.966222
2,2230,Комплекс Санкт-Петербургской одиночной тюрьмы ...,Прачечная и ледник,1885-1890,"Арсенальная наб., 7; Комсомола ул., 8",выявленный объект культурного наследия,Приказ председателя КГИОП № 15 от 20.02.2001,Памятник,POINT (3379975.413 8389295.482),30.362836,59.953064
3,2180,—,"Квартира, в которой жили в 1924-1934 гг. химик...",—,"Академика Лебедева ул., 10-г, кв. 4",объект культурного наследия регионального знач...,Закон Санкт-Петербурга № 174-27 от 05.07.1999,Памятник,POINT (3379068.019 8391163.126),30.354684,59.961463
4,2181,—,Доходный дом Российского общества страхования ...,1912-1914,"Академика Лебедева ул., 12",выявленный объект культурного наследия,Приказ председателя КГИОП № 15 от 20.02.2001,Памятник,POINT (3379054.432 8391248.345),30.354562,59.961847
...,...,...,...,...,...,...,...,...,...,...,...
264,9447,Комплекс зданий Агрофизического научно-исследо...,Главное здание,—,"Гражданский пр., 14, лит. А",выявленный объект культурного наследия,Распоряжение КГИОП от 31.05.2016 № 10-218,Памятник,POINT (3382321.835 8399879.935),30.383914,60.000638
265,9448,Комплекс зданий Агрофизического научно-исследо...,Лаборатория свето-физиологии с оранжереей,—,"Гражданский пр., 14, лит. Д",выявленный объект культурного наследия,Распоряжение КГИОП от 31.05.2016 № 10-218,Памятник,POINT (3382290.777 8399731.656),30.383635,59.999972
266,9534,Кондратьевский жилмассив,Жилые здания,1929-1931,"Санкт-Петербург , Кондратьевский проспект, 40",объект культурного наследия регионального знач...,Распоряжение КГИОП № 10-89 от 27.02.2014,Памятник,POINT (3382123.686 8392992.569),30.382134,59.969689
267,9535,Кондратьевский жилмассив,Здание Калининского универмага,1929-1931,"Санкт-Петербург , Кондратьевский проспект, 40",объект культурного наследия регионального знач...,Распоряжение КГИОП № 10-89 от 27.02.2014,Памятник,POINT (3382476.012 8393048.401),30.385299,59.969940


In [26]:
# TODO сохранить слой в географической проекции в формате GeoJSON
gdf.to_crs(4326).to_file('streets.geojson', driver='GeoJSON')